In [1]:
spark

Waiting for a Spark session to start...

Spark application_1533622723243_14476: Some(http://gw02.itversity.com:4041)

In [2]:
import scala.collection.Map
import scala.collection.mutable.ArrayBuffer
import scala.util.Random
import org.apache.spark.broadcast.Broadcast
import org.apache.spark.ml.recommendation.{ALS, ALSModel}
import org.apache.spark.sql.{DataFrame, Dataset, SparkSession}
import org.apache.spark.sql.functions._

In [30]:
spark.sparkContext.getConf.getAll.foreach(x=>if(x._2.contains("/proxy/")){println(x._2)})

http://rm01.itversity.com:19288/proxy/application_1533622723243_14476


In [34]:
//spark.sparkContext.setCheckpointDir("hdfs:///tmp/")

Name: Syntax Error.
Message: 
StackTrace: 

In [31]:
val base = "hdfs:///user/kranthidr/dataSets/recommendations/audio_profiledata_06-May-2005/"
val rawUserArtistData = spark.read.textFile(base + "user_artist_data.txt")
val rawArtistData = spark.read.textFile(base + "artist_data.txt")
val rawArtistAlias = spark.read.textFile(base + "artist_alias.txt")

base = hdfs:///user/kranthidr/dataSets/recommendations/audio_profiledata_06-May-2005/
rawUserArtistData = [value: string]
rawArtistData = [value: string]
rawArtistAlias = [value: string]


[value: string]

In [32]:
rawUserArtistData.count()

24296858

In [33]:
rawArtistData.count()

1848707

In [35]:
rawArtistAlias.count()

193027

In [36]:
rawUserArtistData.take(5).foreach(println)

1000002 1 55
1000002 1000006 33
1000002 1000007 8
1000002 1000009 144
1000002 1000010 314


In [38]:
val userArtistDF = rawUserArtistData.map { line =>
      val Array(user, artist, _*) = line.split(' ')
      (user.toInt, artist.toInt)
    }.toDF("user", "artist")

userArtistDF = [user: int, artist: int]


[user: int, artist: int]

In [39]:
userArtistDF.agg(min("user"), max("user"), min("artist"), max("artist")).show()

+---------+---------+-----------+-----------+
|min(user)|max(user)|min(artist)|max(artist)|
+---------+---------+-----------+-----------+
|       90|  2443548|          1|   10794401|
+---------+---------+-----------+-----------+



In [40]:
userArtistDF.show(6)

+-------+-------+
|   user| artist|
+-------+-------+
|1000002|      1|
|1000002|1000006|
|1000002|1000007|
|1000002|1000009|
|1000002|1000010|
|1000002|1000013|
+-------+-------+
only showing top 6 rows



In [44]:
userArtistDF.groupBy("user").count().orderBy($"count".desc).show(10)

+-------+-----+
|   user|count|
+-------+-----+
|2030067| 6836|
|1024631| 6188|
|1059334| 5864|
|2016026| 5806|
|2023686| 4863|
|1045464| 4796|
|1004584| 4709|
|2010008| 4512|
|2232770| 4115|
|1006093| 4019|
+-------+-----+
only showing top 10 rows



In [43]:
userArtistDF.groupBy("user").count().orderBy($"count".asc).show(10)

+-------+-----+
|   user|count|
+-------+-----+
|2042404|    1|
|2077194|    1|
|2114054|    1|
|1054207|    1|
|2315849|    1|
|2337539|    1|
|2389464|    1|
|1008233|    1|
|1034317|    1|
|1051364|    1|
+-------+-----+
only showing top 10 rows



In [45]:
rawArtistData.take(5).foreach(println)

1134999	06Crazy Life
6821360	Pang Nakarin
10113088	Terfel, Bartoli- Mozart: Don
10151459	The Flaming Sidebur
6826647	Bodenstandig 3000


In [46]:
rawArtistAlias.take(5).foreach(println)

1092764	1000311
1095122	1000557
6708070	1007267
10088054	1042317
1195917	1042317


In [47]:
 def buildArtistByID(rawArtistData: Dataset[String]): DataFrame = {
    rawArtistData.flatMap { line =>
      val (id, name) = line.span(_ != '\t')
      if (name.isEmpty) {
        None
      } else {
        try {
          Some((id.toInt, name.trim))
        } catch {
          case _: NumberFormatException => None
        }
      }
    }.toDF("id", "name")
  }

  def buildArtistAlias(rawArtistAlias: Dataset[String]): Map[Int,Int] = {
    rawArtistAlias.flatMap { line =>
      val Array(artist, alias) = line.split('\t')
      if (artist.isEmpty) {
        None
      } else {
        Some((artist.toInt, alias.toInt))
      }
    }.collect().toMap
  }

buildArtistByID: (rawArtistData: org.apache.spark.sql.Dataset[String])org.apache.spark.sql.DataFrame
buildArtistAlias: (rawArtistAlias: org.apache.spark.sql.Dataset[String])scala.collection.Map[Int,Int]


In [48]:
val artistByID = buildArtistByID(rawArtistData)
val artistAlias = buildArtistAlias(rawArtistAlias)

artistByID = [id: int, name: string]
artistAlias = Map(1208690 -> 1003926, 2012757 -> 4569, 6949139 -> 1085752, 1109727 -> 1239120, 6772751 -> 1244705, 2070533 -> 1021544, 1157679 -> 2194, 9969617 -> 5630, 2034496 -> 1116214, 6764342 -> 40, 1272489 -> 1278238, 2108744 -> 1009267, 10349857 -> 1000052, 2145319 -> 1020463, 2126338 -> 2717, 10165456 -> 1001169, 6779368 -> 1239506, 10278137 -> 1001523, 9939075 -> 1329390, 2037201 -> 1274155, 1248585 -> 2885, 1106945 -> 1399, 6811322 -> 1019016, 9978396 -> 1784, 6676961 -> 1086433, 2117821 -> 2611, 6863616 -> 1277013, 6895480 -> 1000993, 6831632 -> 1246136, 1001719 -> 1009727, 10135633 -> 4250, 7029291 -> 1034635, 6967939 -> 1002734, 6864694 -> 1017311, 1237279 -> 1029752, 6793956...


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [49]:
val (badID, goodID) = artistAlias.head

badID = 1208690
goodID = 1003926


1003926

In [50]:
artistByID.filter($"id" isin (badID, goodID)).show()

+-------+----------------+
|     id|            name|
+-------+----------------+
|1208690|Collective Souls|
|1003926| Collective Soul|
+-------+----------------+



In [51]:
def buildCounts(
      rawUserArtistData: Dataset[String],
      bArtistAlias: Broadcast[Map[Int,Int]]): DataFrame = {
    rawUserArtistData.map { line =>
      val Array(userID, artistID, count) = line.split(' ').map(_.toInt)
      val finalArtistID = bArtistAlias.value.getOrElse(artistID, artistID)
      (userID, finalArtistID, count)
    }.toDF("user", "artist", "count")
  }

buildCounts: (rawUserArtistData: org.apache.spark.sql.Dataset[String], bArtistAlias: org.apache.spark.broadcast.Broadcast[scala.collection.Map[Int,Int]])org.apache.spark.sql.DataFrame


In [52]:
val bArtistAlias = spark.sparkContext.broadcast(buildArtistAlias(rawArtistAlias))
val trainData = buildCounts(rawUserArtistData, bArtistAlias).cache()

bArtistAlias = Broadcast(36)
trainData = [user: int, artist: int ... 1 more field]


[user: int, artist: int ... 1 more field]

In [53]:
trainData.show(5)

+-------+-------+-----+
|   user| artist|count|
+-------+-------+-----+
|1000002|      1|   55|
|1000002|1000006|   33|
|1000002|1000007|    8|
|1000002|1000009|  144|
|1000002|1000010|  314|
+-------+-------+-----+
only showing top 5 rows



In [54]:
 val model = new ALS().
      setSeed(Random.nextLong()).
      setImplicitPrefs(true).
      setRank(10).
      setRegParam(0.01).
      setAlpha(1.0).
      setMaxIter(5).
      setUserCol("user").
      setItemCol("artist").
      setRatingCol("count").
      setPredictionCol("prediction").
      fit(trainData)

model = als_9b43b41b34ce


als_9b43b41b34ce

In [58]:
model.userFactors.show(1,false)

+---+------------------------------------------------------------------------------------------------------------------------+
|id |features                                                                                                                |
+---+------------------------------------------------------------------------------------------------------------------------+
|90 |[0.23245662, -0.089350305, 0.4406498, -0.52033013, 1.1043696, 0.15526487, 0.34216267, 0.40751842, 0.1901746, -0.6059311]|
+---+------------------------------------------------------------------------------------------------------------------------+
only showing top 1 row



In [60]:
 model.userFactors.select("features").show(2,truncate = false)

+------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                |
+------------------------------------------------------------------------------------------------------------------------+
|[0.23245662, -0.089350305, 0.4406498, -0.52033013, 1.1043696, 0.15526487, 0.34216267, 0.40751842, 0.1901746, -0.6059311]|
|[0.2145854, 0.08403039, -0.0794857, 0.2887465, 0.18284331, 0.10450412, -0.0195537, 0.10427659, -0.15771231, -0.06480175]|
+------------------------------------------------------------------------------------------------------------------------+
only showing top 2 rows



In [71]:
 val toRecommend = model.itemFactors.
      select($"id".as("artist")).
      withColumn("user", lit(userID))

toRecommend = [artist: int, user: int]


lastException: Throwable = null


[artist: int, user: int]

In [75]:
toRecommend.count()

1568126

In [72]:
toRecommend.show(5,false)

+------+-------+
|artist|user   |
+------+-------+
|30    |2093760|
|40    |2093760|
|50    |2093760|
|70    |2093760|
|90    |2093760|
+------+-------+
only showing top 5 rows



In [73]:
model.transform(toRecommend).show(5,false)

Name: org.apache.spark.sql.AnalysisException
Message: Detected cartesian product for LEFT OUTER join between logical plans
Project [_1#283 AS artist#407]
+- SerializeFromObject [assertnotnull(input[0, scala.Tuple2, true])._1 AS _1#283, staticinvoke(class org.apache.spark.sql.catalyst.expressions.UnsafeArrayData, ArrayType(FloatType,false), fromPrimitiveArray, assertnotnull(input[0, scala.Tuple2, true])._2, true, false) AS _2#284]
   +- ExternalRDD [obj#282]
and
Project [_2#274 AS features#277]
+- Filter (UDF(2093760) = _1#273)
   +- SerializeFromObject [assertnotnull(input[0, scala.Tuple2, true])._1 AS _1#273, staticinvoke(class org.apache.spark.sql.catalyst.expressions.UnsafeArrayData, ArrayType(FloatType,false), fromPrimitiveArray, assertnotnull(input[0, scala.Tuple2, true])._2, true, false) AS _2#274]
      +- ExternalRDD [obj#272]
Join condition is missing or trivial.
Use the CROSS JOIN syntax to allow cartesian products between these relations.;
StackTrace: Project [_1#283 AS arti

In [61]:
def makeRecommendations(model: ALSModel, userID: Int, howMany: Int): DataFrame = {
    val toRecommend = model.itemFactors.
      select($"id".as("artist")).
      withColumn("user", lit(userID))
    
    model.transform(toRecommend).
      select("artist", "prediction").
      orderBy($"prediction".desc).
      limit(howMany)
  }

makeRecommendations: (model: org.apache.spark.ml.recommendation.ALSModel, userID: Int, howMany: Int)org.apache.spark.sql.DataFrame


In [62]:
val userID = 2093760

val existingArtistIDs = trainData.
      filter($"user" === userID).
      select("artist").as[Int].collect()

userID = 2093760
existingArtistIDs = Array(1180, 1255340, 378, 813, 942)


[1180, 1255340, 378, 813, 942]

In [69]:
artistByID.filter($"id" isin (existingArtistIDs:_*)).show()

+-------+---------------+
|     id|           name|
+-------+---------------+
|   1180|     David Gray|
|    378|  Blackalicious|
|    813|     Jurassic 5|
|1255340|The Saw Doctors|
|    942|         Xzibit|
+-------+---------------+



In [70]:
val topRecommendations = makeRecommendations(model, userID, 5)

topRecommendations.show()

Name: org.apache.spark.sql.AnalysisException
Message: Detected cartesian product for LEFT OUTER join between logical plans
Project [_1#283 AS artist#357]
+- SerializeFromObject [assertnotnull(input[0, scala.Tuple2, true])._1 AS _1#283, staticinvoke(class org.apache.spark.sql.catalyst.expressions.UnsafeArrayData, ArrayType(FloatType,false), fromPrimitiveArray, assertnotnull(input[0, scala.Tuple2, true])._2, true, false) AS _2#284]
   +- ExternalRDD [obj#282]
and
Project [_2#274 AS features#277]
+- Filter (UDF(2093760) = _1#273)
   +- SerializeFromObject [assertnotnull(input[0, scala.Tuple2, true])._1 AS _1#273, staticinvoke(class org.apache.spark.sql.catalyst.expressions.UnsafeArrayData, ArrayType(FloatType,false), fromPrimitiveArray, assertnotnull(input[0, scala.Tuple2, true])._2, true, false) AS _2#274]
      +- ExternalRDD [obj#272]
Join condition is missing or trivial.
Use the CROSS JOIN syntax to allow cartesian products between these relations.;
StackTrace: Project [_1#283 AS arti

In [ ]:
val recommendedArtistIDs = topRecommendations.select("artist").as[Int].collect()

artistByID.filter($"id" isin (recommendedArtistIDs:_*)).show()

In [ ]:
def areaUnderCurve(
      positiveData: DataFrame,
      bAllArtistIDs: Broadcast[Array[Int]],
      predictFunction: (DataFrame => DataFrame)): Double = {

    // What this actually computes is AUC, per user. The result is actually something
    // that might be called "mean AUC".

    // Take held-out data as the "positive".
    // Make predictions for each of them, including a numeric score
    val positivePredictions = predictFunction(positiveData.select("user", "artist")).
      withColumnRenamed("prediction", "positivePrediction")

    // BinaryClassificationMetrics.areaUnderROC is not used here since there are really lots of
    // small AUC problems, and it would be inefficient, when a direct computation is available.

    // Create a set of "negative" products for each user. These are randomly chosen
    // from among all of the other artists, excluding those that are "positive" for the user.
    val negativeData = positiveData.select("user", "artist").as[(Int,Int)].
      groupByKey { case (user, _) => user }.
      flatMapGroups { case (userID, userIDAndPosArtistIDs) =>
        val random = new Random()
        val posItemIDSet = userIDAndPosArtistIDs.map { case (_, artist) => artist }.toSet
        val negative = new ArrayBuffer[Int]()
        val allArtistIDs = bAllArtistIDs.value
        var i = 0
        // Make at most one pass over all artists to avoid an infinite loop.
        // Also stop when number of negative equals positive set size
        while (i < allArtistIDs.length && negative.size < posItemIDSet.size) {
          val artistID = allArtistIDs(random.nextInt(allArtistIDs.length))
          // Only add new distinct IDs
          if (!posItemIDSet.contains(artistID)) {
            negative += artistID
          }
          i += 1
        }
        // Return the set with user ID added back
        negative.map(artistID => (userID, artistID))
      }.toDF("user", "artist")

    // Make predictions on the rest:
    val negativePredictions = predictFunction(negativeData).
      withColumnRenamed("prediction", "negativePrediction")

    // Join positive predictions to negative predictions by user, only.
    // This will result in a row for every possible pairing of positive and negative
    // predictions within each user.
    val joinedPredictions = positivePredictions.join(negativePredictions, "user").
      select("user", "positivePrediction", "negativePrediction").cache()

    // Count the number of pairs per user
    val allCounts = joinedPredictions.
      groupBy("user").agg(count(lit("1")).as("total")).
      select("user", "total")
    // Count the number of correctly ordered pairs per user
    val correctCounts = joinedPredictions.
      filter($"positivePrediction" > $"negativePrediction").
      groupBy("user").agg(count("user").as("correct")).
      select("user", "correct")

    // Combine these, compute their ratio, and average over all users
    val meanAUC = allCounts.join(correctCounts, Seq("user"), "left_outer").
      select($"user", (coalesce($"correct", lit(0)) / $"total").as("auc")).
      agg(mean("auc")).
      as[Double].first()

    joinedPredictions.unpersist()

    meanAUC
  }

In [ ]:
val allData = buildCounts(rawUserArtistData, bArtistAlias)
val Array(trainData, cvData) = allData.randomSplit(Array(0.9, 0.1))
trainData.cache()
cvData.cache()

In [ ]:
val allArtistIDs = allData.select("artist").as[Int].distinct().collect()
val bAllArtistIDs = spark.sparkContext.broadcast(allArtistIDs)

In [ ]:
val model = new ALS().
          setSeed(Random.nextLong()).
          setImplicitPrefs(true).
          setRank(10).setRegParam(0.01).
          setAlpha(1.0).setMaxIter(5).
          setUserCol("user").setItemCol("artist").
          setRatingCol("count").setPredictionCol("prediction").
          fit(trainData)

In [ ]:
areaUnderCurve(cvData, bAllArtistIDs, model.transform)

In [ ]:
def predictMostListened(train: DataFrame)(allData: DataFrame): DataFrame = {
    val listenCounts = train.groupBy("artist").
      agg(sum("count").as("prediction")).
      select("artist", "prediction")
    allData.
      join(listenCounts, Seq("artist"), "left_outer").
      select("user", "artist", "prediction")
  }

In [ ]:
val mostListenedAUC = areaUnderCurve(cvData, bAllArtistIDs, predictMostListened(trainData))
println(mostListenedAUC)

In [ ]:
val evaluations =
      for (rank     <- Seq(5,  30);
           regParam <- Seq(1.0, 0.0001);
           alpha    <- Seq(1.0, 40.0))
      yield {
        val model = new ALS().
          setSeed(Random.nextLong()).
          setImplicitPrefs(true).
          setRank(rank).setRegParam(regParam).
          setAlpha(alpha).setMaxIter(20).
          setUserCol("user").setItemCol("artist").
          setRatingCol("count").setPredictionCol("prediction").
          fit(trainData)

        val auc = areaUnderCurve(cvData, bAllArtistIDs, model.transform)

        model.userFactors.unpersist()
        model.itemFactors.unpersist()

        (auc, (rank, regParam, alpha))
      }

In [ ]:
evaluations.sorted.reverse.foreach(println)

In [ ]:
val someUsers = allData.select("user").as[Int].distinct().take(25)

In [ ]:
val someRecommendations = someUsers.map(userID => (userID, makeRecommendations(model, userID, 5)))

In [ ]:
someRecommendations.foreach{
    
case(userID, recsDF) => 
    
val recommendedArtists = recsDF.select("artist").as[Int].collect()
    
printn(s"$userID -> ${recommendedArtists.mkString(", ")}")

}